In [2]:
import numpy as np
import pandas as pd
import requests 
import math
!pip install xlsxwriter
import xlsxwriter
from google.colab import files
import io

uploaded = files.upload()

#IEX_API_TOKEN
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

#IMPORT STOCK LIST
stocks = pd.read_csv(io.BytesIO(uploaded['sp_500_stocks.csv']))





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving sp_500_stocks.csv to sp_500_stocks (1).csv


In [9]:

IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
#Adding Our Stocks Data to a Pandas DataFrame
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

def chunks(lst, n):
  #yeilds succesive n sized chunks from lists
  for i in range (0,len(lst),n):
    yield lst[i: i+n]

symbol_groups= list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

for symbol_string in symbol_strings:
  batch_api_call_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    if symbol=='DISCA' or symbol=='HFC' or symbol=='VIAC' or symbol=='WLTW':
      continue
    final_dataframe = final_dataframe.append(
                                    pd.Series([symbol, 
                                                data[symbol]['quote']['latestPrice'], 
                                                data[symbol]['quote']['marketCap'], 
                                                'N/A'], 
                                              index = my_columns), 
                                    ignore_index = True)
print(final_dataframe)


    Ticker   Price Market Capitalization Number Of Shares to Buy
0        A  125.78           38380436633                     N/A
1      AAL   14.35            9261443954                     N/A
2      AAP  193.57           11703895978                     N/A
3     AAPL  159.60         2603713588348                     N/A
4     ABBV  151.03          267527099304                     N/A
..     ...     ...                   ...                     ...
496    YUM  122.30           35013797807                     N/A
497    ZBH  108.00           22535782329                     N/A
498   ZBRA  327.20           17408410323                     N/A
499   ZION   54.74            8384646156                     N/A
500    ZTS  179.01           88000245917                     N/A

[501 rows x 4 columns]


In [10]:
#Calculating the number of shares to buy

portfolio_size=input('Enter the value of your portfolio: ')

try:
  val=float(portfolio_size)

except ValueError:
  print("Please enter an integer value thats not an integer")
  portfolio_size=input('Enter the value of your portfolio: ')
  val=float(portfolio_size)


Enter the value of your portfolio: 1000000


In [13]:
position_size= val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i,'Number Of Shares to Buy']= math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,125.78,38380436633,15
1,AAL,14.35,9261443954,139
2,AAP,193.57,11703895978,10
3,AAPL,159.60,2603713588348,12
4,ABBV,151.03,267527099304,13
...,...,...,...,...
496,YUM,122.30,35013797807,16
497,ZBH,108.00,22535782329,18
498,ZBRA,327.20,17408410323,6
499,ZION,54.74,8384646156,36


In [12]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)
writer.save()

# New Section